In [ ]:
import os
import subprocess
from pathlib import Path
import torch
from mmdet3d.evaluation.metrics import nuscenes_metric as nus_metric
from mmdet3d.evaluation.metrics.nuscenes_metric import output_to_nusc_box
import json 
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import transform_matrix
import operator
from functools import reduce
from pathlib import Path
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.eval.detection.config import config_factory
from nuscenes.eval.detection.evaluate import NuScenesEval
from nuscenes.utils.data_classes import RadarPointCloud
from classes import cls_attr_dist, class_names, mini_val_tokens
from custom_env import home_dir, output_dir, preds_dir, model_dir, is_set_to_mini
from custom_env import dataset_root as dataroot
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility

import nuscenes_render

In [ ]:
eval_set_map = {
        'v1.0-mini': 'mini_val',
        'v1.0-trainval': 'val',
        'v1.0-test': 'test'
    }

dataset_version = 'v1.0-mini' if is_set_to_mini() else 'v1.0-trainval'
try:
    eval_version = 'detection_cvpr_2019'
    eval_config = config_factory(eval_version)
except:
    eval_version = 'cvpr_2019'
    eval_config = config_factory(eval_version)
    

DETECTION_THRESHOLD = 0.35

backend_args = None
nusc = NuScenes(version=dataset_version, dataroot = dataroot)
ann_file = f'{dataroot}nuscenes_infos_val.pkl'
metric='bbox'

pcd_path = f"{dataroot}/samples/LIDAR_TOP/"
mmdet_path = f"{home_dir}/software/mmdetection3d"
pcd_list = os.listdir(pcd_path)

In [ ]:
def __load_ego_veh(self, sample_token:str):
    sample = nusc.get('sample', sample_token)
    sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
    return nusc.get('ego_pose', sd_record['ego_pose_token'])

print(__load_ego_veh(nusc, "b5989651183643369174912bc5641d3b"))

In [ ]:
import random

sample = "b5989651183643369174912bc5641d3b"
sample = nusc.get('sample', sample)

In [ ]:
# sd_record = nusc.get('sample', sample['data'])
# sd_record
for token in sample['data']:
    sd_record = nusc.get('sample_data', sample['data'][token])
    print(sd_record['filename'])


In [ ]:
sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
sd_record['filename']

In [ ]:
nusc.get('ego_pose', sd_record['ego_pose_token'])

In [ ]:
from pyquaternion import Quaternion
sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
pose = nusc.get('ego_pose', sd_record['ego_pose_token'])
q = Quaternion(pose['rotation'])

In [ ]:
import math
math.atan2(pose['translation'][1], pose['translation'][0])

In [ ]:
q.yaw_pitch_roll

psi = q.yaw_pitch_roll[0]

psi/np.pi * 180

In [ ]:
from nuscenes.eval.common.utils import quaternion_yaw
# PREFER THIS
rotation = Quaternion(pose['rotation'])
yaw = quaternion_yaw(rotation) / np.pi * 180

yaw

In [ ]:
import random

sample = "b5989651183643369174912bc5641d3b"
sample = nusc.get('sample', sample)

nuscenes_render.render_sample_data_with_predictions(sample["data"]["LIDAR_TOP"], nusc=nusc)

# Figure out the coordinate frame
# Plot the yaw on the map below

In [ ]:
# nusc.render_sample("b5989651183643369174912bc5641d3b")

In [ ]:
scene = nusc.scene[0]
scene

In [ ]:
def get_ego_pose(token):
    for ego_pose in nusc.ego_pose:
        if ego_pose['token'] == token:
            return ego_pose
    print("Ego pose not found for token", token)
        
get_ego_pose(sample['token'])

In [ ]:
sample = "b5989651183643369174912bc5641d3b"
sample = nusc.get('sample', sample)

a, boxes, b = nusc.get_sample_data(sample["data"]["LIDAR_TOP"], 
                     box_vis_level=BoxVisibility.ANY,
                     use_flat_vehicle_coordinates=True)

np.degrees(np.arctan2(boxes[0].center[1], boxes[0].center[0]))

In [ ]:
nusc.get('ego_pose', nusc.get('sample', sample['token'])['ego_pose_token'])

In [ ]:
np.array(boxes[0].center[:2])

In [ ]:
from cluster import RadiusBand

r = RadiusBand(
                 sample_token = "a",
                 gt_boxes=[], 
                 ego_veh:{},
                 radius_band:(0, 11),
                 max_distance_bw_obj:10)

SyntaxError: invalid syntax (4260365361.py, line 6)